## Import libary và datasets

In [1]:
# Install libs
! pip install -qq peft==0.15.2 trl==0.17.0 
# bitsandbytes==0.45.5
! pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires

#### Thêm hàng này để tránh chạy multi gpu gây lỗi: RuntimeError: Input tensors need to be on the same GPU, but found the following tensor and device combinations (thêm trước khi import thư viện)

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
print("CUDA devices =", torch.cuda.device_count())   # Phải in ra 1
print("Current device =", torch.cuda.current_device())  # Phải là 0

CUDA devices = 1
Current device = 0


In [3]:
from trl import DPOTrainer, DPOConfig
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import wandb

from peft import LoraConfig, get_peft_model
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Preference Tuning with LLMs")
secret_value_1 = user_secrets.get_secret("push_to_hub")

wandb.login(key=secret_value_0)

2025-05-09 08:43:13.056180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746780193.292445      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746780193.361109      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: moonlig73 (minhdeptrai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
CACHE_DIR = "/kaggle/working/"

dataset = load_dataset("thainq107/Vi-Alpaca-Preference", cache_dir=CACHE_DIR)
dataset_train = dataset["train"].select(range(50))
dataset_train[0]

README.md:   0%|          | 0.00/506 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/61.9M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.95M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65017 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'id': 'alpaca-7294',
 'question': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.',
 'chosen': 'Không có lỗi ngữ pháp. Câu này đã chính xác.',
 'rejected': 'Câu này không có lỗi ngữ pháp.'}

## Initialize model và tokenizer

In [5]:
LORA_RANK        = 8
TARGET_MODULES   = [
    "q_proj","k_proj","v_proj","o_proj",
    "gate_proj","up_proj","down_proj",
]
MAX_SEQ          = 1024
peft_config = LoraConfig(
    r=LORA_RANK,
    lora_alpha=LORA_RANK,
    target_modules=TARGET_MODULES,
    task_type="CAUSAL_LM",
)

In [6]:
model_name = "TSunm/Llama-3.2-1B-Instruct-Chat-sft"
cache_dir = "/kaggle/working/"

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto", cache_dir=cache_dir,
)

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

## Preprocessing

In [7]:
def convert_to_conversational_preference_format(example):
    return {
        "id": example["id"],
        "prompt": [{"role": "system",
                    "content": "You are a helpful assistant."},
                   {"role": "user",
                    "content": example["question"]}],
        "chosen": [{"role": "assistant",
                    "content": example["chosen"]}],
        "rejected": [{"role": "assistant",
                      "content": example["rejected"]}],
    }

dpo_dataset = dataset_train.map(convert_to_conversational_preference_format)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

## Trainning

In [8]:
hyperparameters = {
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 8,
    "gradient_checkpointing": True,
    "learning_rate": 3e-5,
    "logging_steps": 200,
    "num_train_epochs": 1,
    "save_strategy": "no",
    "overwrite_output_dir": True,
    "optim": "paged_adamw_8bit",
    "warmup_steps": 200,
    "fp16": True
}

In [9]:
DPO_OUTPUT_DIR = f"/kaggle/working/Llama-3.2-1B-Instruct-Chat-DPO"
dpo_args = DPOConfig(
    **{ **hyperparameters, "output_dir":
       DPO_OUTPUT_DIR, "max_length": MAX_SEQ }
)

dpo_trainer = DPOTrainer(
    base_model,
    args=dpo_args,
    train_dataset=dpo_dataset,
    processing_class=tokenizer,
    peft_config=peft_config,
)
dpo_trainer.train()

Extracting prompt in train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250509_084410-nhv0iry4
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run /kaggle/working/Llama-3.2-1B-Instruct-Chat-DPO
wandb: ⭐️ View project at https://wandb.ai/minhdeptrai/huggingface
wandb: 🚀 View run at https://wandb.ai/minhdeptrai/huggingface/runs/nhv0iry4
`use_cache=True` is inco

Step,Training Loss


TrainOutput(global_step=3, training_loss=0.6930636564890543, metrics={'train_runtime': 29.9002, 'train_samples_per_second': 1.672, 'train_steps_per_second': 0.1, 'total_flos': 0.0, 'train_loss': 0.6930636564890543, 'epoch': 0.96})

## Push to hub and inference

In [10]:
from huggingface_hub import HfApi
model_name = "TSunm/Llama-3.2-1B-Instruct-Chat-DPO"

api = HfApi()
api.create_repo(
    repo_id=model_name,
    token=secret_value_1,
    repo_type="model",
    exist_ok=True 
)

RepoUrl('https://huggingface.co/TSunm/Llama-3.2-1B-Instruct-Chat-DPO', endpoint='https://huggingface.co', repo_type='model', repo_id='TSunm/Llama-3.2-1B-Instruct-Chat-DPO')

In [11]:
dpo_trainer.model.push_to_hub(
    repo_id=model_name,
    token=secret_value_1
)
# push tokenizer lên cùng
tokenizer.push_to_hub(
    repo_id=model_name,
    token=secret_value_1
)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/22.6M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/TSunm/Llama-3.2-1B-Instruct-Chat-DPO/commit/3bf32e1450292d52111b1bdf3c24e8e807917397', commit_message='Upload tokenizer', commit_description='', oid='3bf32e1450292d52111b1bdf3c24e8e807917397', pr_url=None, repo_url=RepoUrl('https://huggingface.co/TSunm/Llama-3.2-1B-Instruct-Chat-DPO', endpoint='https://huggingface.co', repo_type='model', repo_id='TSunm/Llama-3.2-1B-Instruct-Chat-DPO'), pr_revision=None, pr_num=None)